In [82]:
#default_exp parser
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
#export
from pathlib import Path
import re
import xml.etree.ElementTree as et
from zipfile import ZipFile
from datetime import datetime
import pandas as pd
import pandas_read_xml as pdx
from anateldb.query import *

In [84]:
#export
def read_stel(pasta, update=False):
    """Lê o banco de dados salvo localmente do STEL. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01."""
    if update:
        update_stel(pasta)
    if (file := Path(f'{pasta}/stel.fth')).exists():
        stel = pd.read_feather(file)
    elif (file := Path(f'{pasta}/stel.csv')).exists():
        stel = pd.read_csv(file)
    elif (file := Path(f'{pasta}/Base_de_Dados.xlsx')).exists():
        stel = pd.read_excel(file, sheet_name='Stel', engine='openpyxl')
    else:
        update_stel(pasta)
        try:
            stel = pd.read_feather(Path(f'{pasta}/stel.fth'))
        except FileNotFoundError as e:
            raise ConnectionError("Base de Dados do Stel inexistente e não foi possível atualizá-la" ) from e
    return stel

In [85]:
#export
def read_radcom(pasta, update=False):
    """Lê o banco de dados salvo localmente de RADCOM. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01."""
    if update:
        update_radcom(pasta)
    if (file := Path(f'{pasta}/radcom.fth')).exists():
        radcom = pd.read_feather(file)
    elif (file := Path(f'{pasta}/radcom.csv')).exists():
        radcom = pd.read_csv(file)
    elif (file := Path(f'{pasta}/Base_de_Dados.xlsx')).exists():
         radcom = pd.read_excel(file, sheet_name='Radcom', engine='openpyxl')
    else:
        update_radcom(pasta)
        try:
            radcom = pd.read_feather(Path(f'{pasta}/radcom.fth'))
        except FileNotFoundError as e:
            raise ConnectionError("Base de Dados do Stel inexistente e não foi possível atualizá-la" ) from e
    return radcom

In [86]:
#export
def parse_estações(row, cols=COL_ESTACOES):
    """Receives a json row and filter the column in `cols`"""
    dict_result = {k: row[k] for k in cols}
    dict_result['Número da Estação'] = row['administrativo']['@numero_estacao']
    return dict_result

def parse_plano_basico(row, cols=COL_PLANO_BASICO):
    """Receives a json row and filter the column in `cols`"""
    if '@Frequencia' not in row:
        cols = list(cols)
        cols[cols.index('@Frequencia')] = '@Frequência'
    return {k: row[k] for k in cols}

def read_estações(path, cols):
    df = pdx.read_xml(path, cols)
    columns = ['Serviço', 'Situação', 'Entidade', 'Fistel', 'CNPJ', 'Município', 'UF', 'Id', 'Número da Estação']
    dtypes = ['category', 'category', 'str', 'str', 'str', 'str', 'category', 'str', 'str']
    df = pd.DataFrame(df['row'].apply(parse_estações).tolist())
    df.columns = columns
    for col, dtype in zip(columns, dtypes):
        df[col] = df[col].astype(dtype)
    return df

def read_plano_basico(path, cols):
    df = pdx.read_xml(path, cols)
    columns = ['Id', 'Classe', 'Frequência', 'Latitude', 'Longitude']
    dtypes = ['str', 'category', 'str', 'str', 'str']
    df = pd.DataFrame(df['row'].apply(parse_plano_basico).tolist())
    df.loc[df['@Frequencia'].isna(), '@Frequencia'] = df.loc[df['@Frequência'].notna(), '@Frequência']
    df.drop('@Frequência', axis=1, inplace=True)
    df.columns = columns
    for col, dtype in zip(columns, dtypes):
        df[col] = df[col].astype(dtype)
    return df

def read_historico(path):
    regex = r"\s([a-zA-Z]+)=\'{1}([\w\-\ :\.]*)\'{1}"
    with ZipFile(path) as xmlzip:
        with xmlzip.open('documento_historicos.xml', 'r') as xml:
            xml_list = xml.read().decode().split('\n')[2:-1]
    dict_list = []
    for item in xml_list:
        matches = re.finditer(regex, item, re.MULTILINE)
        dict_list.append(dict(match.groups() for match in matches))
    df = pd.DataFrame(dict_list)
    df = df[(df.tipoDocumento == 'Ato') & (df.razao == 'Autoriza o Uso de Radiofrequência')].reset_index()
    df = df.loc[:, ['id', 'numeroDocumento', 'orgao', 'dataDocumento']]
    df.columns = ['Id', 'Num_Ato', 'Órgao', 'Data_Ato']
    df['Data_Ato'] = pd.to_datetime(df.Data_Ato)    
    return df.sort_values('Data_Ato').groupby('Id').last().reset_index()

In [87]:
#export
def read_mosaico(pasta, update=False):
    if update:
        update_mosaico(pasta)
        estações = read_estações(f'{pasta}/estações.zip', ['estacao_rd'])
        plano_basico = read_plano_basico(f'{pasta}/plano_basico.zip', ['plano_basico'])
        historico = read_historico(f'{pasta}/historico.zip')
        df = pd.merge(estações, plano_basico, on='Id').merge(historico, on='Id')
        df.to_feather(f'{pasta}/mosaico.fth')
        return df
    if not (file := Path(f'{pasta}/mosaico.fth')).exists():
            return read_mosaico(pasta, update=True)
    return pd.read_feather(f'{pasta}/mosaico.fth')

In [88]:
from nbdev.export import notebook2script; notebook2script()

Converted filter.ipynb.
Converted index.ipynb.
Converted parser.ipynb.
Converted queries.ipynb.
